In [38]:

#set year: 2023, 24, 25
year = 2025

In [39]:
import pandas as pd

#we load the list of distinct tpurnaments per year.
#We got this from SQL query at "query_tournaments"
df_tornei = pd.read_csv(f"tornei/tornei_{str(year)[-2:]}.csv")
df_iscrizioni = pd.read_csv(f"df_iscrizioni_player_event/df_{year}.csv")

players = df_iscrizioni["PlayerCode"].dropna().unique()
events  = df_tornei["EventId"].dropna().unique()

grid = (    pd.MultiIndex.from_product([players, events], names=["PlayerCode", "EventId"]).to_frame(index=False))

iscr_key = df_iscrizioni[["PlayerCode", "EventId","PointsGained"]].drop_duplicates()
iscr_key["iscritto"] = 1
out = grid.merge(iscr_key, on=["PlayerCode", "EventId"], how="left")
out["iscritto"] = out["iscritto"].fillna(0).astype(int)

out["PointsGained"] = out["PointsGained"].fillna(0).astype(int)


out = out.merge(df_tornei, on="EventId", how="left")

display(out)


,PlayerCode,EventId,PointsGained,iscritto,year,date_tournament,EventName,EventType,EventCountry,Surface,TotPrizeMoney
0,s0re,747,0,1,2025,2025-09-22,Beijing,500,China,Hard,4016050
1,s0re,7581,25,1,2025,2025-09-15,Chengdu,250,China,Hard,1190210
2,s0re,336,100,1,2025,2024-12-30,Hong Kong,250,Hong Kong,Hard,680140
3,s0re,6242,0,0,2025,2025-08-18,Winston-Salem,250,"NC, U.S.A.",Hard,798335
4,s0re,319,0,0,2025,2025-07-21,Kitzbuhel,250,Austria,Clay,596035
...,...,...,...,...,...,...,...,...,...,...,...
16831,n0db,425,0,0,2025,2025-04-14,Barcelona,500,Spain,Clay,2889200
16832,n0db,337,0,0,2025,2025-10-20,Vienna,500,Austria,Hard,2736875
16833,n0db,8996,0,0,2025,2025-02-24,Santiago,250,Chile,Clay,680140
16834,n0db,717,0,0,2025,2025-03-31,Houston,250,"TX, U.S.A.",Clay,680140


In [40]:
#we add a column that tells if the player's nationality is the same as the tournament country
tornei = pd.read_csv(f"df_iscrizioni_player_event/df_{year}.csv")
players = pd.read_csv(f"players_list/atp_senza_ripetizioni_{year}.csv")

tornei = tornei[["PlayerCode", "EventId", "EventCountry"]]
players = players[["Player Id", "Nationality"]]

df = tornei.merge(
    players,
    left_on="PlayerCode",
    right_on="Player Id",
    how="left"
)

#we load the geo data for country codes from a github repo: "https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv"
#this because the players' nationalities are in alpha-3 code, while the tournament countries are in full name
geo1 = pd.read_csv("geografia.csv")

df_primo_join = df.merge(geo1,
    left_on="EventCountry",
    right_on="name",
    how="left"
)

df_primo_join["Nationality"] = df_primo_join["Nationality"].str.upper()


df_finale = df_primo_join[df_primo_join["Nationality"] == df_primo_join["alpha-3"]]
df_finale=df_finale[["PlayerCode", "EventId", "EventCountry","Nationality"]]
df_finale

,PlayerCode,EventId,EventCountry,Nationality
0,s0re,5014,China,CHN
1,s0re,747,China,CHN
2,s0re,7581,China,CHN
13,bd06,8994,Spain,ESP
19,bd06,1536,Spain,ESP
...,...,...,...,...
9632,b0bu,7009,France,FRA
9637,b0bu,2973,France,FRA
9639,b0bu,9162,France,FRA
9641,b0bu,7874,France,FRA


In [41]:
# add column same_n
out["Same_Nationality"] = 0

for index, row in out.iterrows():
    player_code = row["PlayerCode"]
    event_id = row["EventId"]
    
    match = df_finale[
        (df_finale["PlayerCode"] == player_code) & 
        (df_finale["EventId"] == event_id)
    ]
    
    if not match.empty:
        out.at[index, "Same_Nationality"] = 1
out

,PlayerCode,EventId,PointsGained,iscritto,year,date_tournament,EventName,EventType,EventCountry,Surface,TotPrizeMoney,Same_Nationality
0,s0re,747,0,1,2025,2025-09-22,Beijing,500,China,Hard,4016050,1
1,s0re,7581,25,1,2025,2025-09-15,Chengdu,250,China,Hard,1190210,1
2,s0re,336,100,1,2025,2024-12-30,Hong Kong,250,Hong Kong,Hard,680140,0
3,s0re,6242,0,0,2025,2025-08-18,Winston-Salem,250,"NC, U.S.A.",Hard,798335,0
4,s0re,319,0,0,2025,2025-07-21,Kitzbuhel,250,Austria,Clay,596035,0
...,...,...,...,...,...,...,...,...,...,...,...,...
16831,n0db,425,0,0,2025,2025-04-14,Barcelona,500,Spain,Clay,2889200,0
16832,n0db,337,0,0,2025,2025-10-20,Vienna,500,Austria,Hard,2736875,0
16833,n0db,8996,0,0,2025,2025-02-24,Santiago,250,Chile,Clay,680140,0
16834,n0db,717,0,0,2025,2025-03-31,Houston,250,"TX, U.S.A.",Clay,680140,0


In [42]:
out.to_csv(f"df_per_aggiungere_pts_def/df_{year}.csv", index=False)

#binary variable PTS_to_defend is then added with a SQL query at "query_aggiunti_pts_def"
#the resulting df are stored in "df_pre_dummies" folder with the final df used for regression.